In [1]:
from app import NotionAPI

In [2]:
notion = NotionAPI()
data = notion.query_db()
data

{'object': 'list',
 'results': [{'object': 'page',
   'id': '16782441-9775-46fe-b2f9-e7971473044f',
   'created_time': '2021-12-07T18:56:00.000Z',
   'last_edited_time': '2021-12-07T18:58:00.000Z',
   'cover': None,
   'icon': None,
   'parent': {'type': 'database_id',
    'database_id': 'aa1f8361-5ff9-4523-9302-887264317370'},
   'archived': False,
   'properties': {'ISIN': {'id': '%40%5BZb',
     'type': 'rich_text',
     'rich_text': []},
    'Tipo': {'id': 'IUV%7D',
     'type': 'select',
     'select': {'id': '4e0531e5-b001-4d03-ac9e-9054c79ee931',
      'name': 'Ingreso',
      'color': 'purple'}},
    'Descripción': {'id': 'MnyC',
     'type': 'rich_text',
     'rich_text': [{'type': 'text',
       'text': {'content': 'Flatex Deposit', 'link': None},
       'annotations': {'bold': False,
        'italic': False,
        'strikethrough': False,
        'underline': False,
        'code': False,
        'color': 'default'},
       'plain_text': 'Flatex Deposit',
       'href': Non

In [5]:
data

{'object': 'list',
 'results': [{'object': 'page',
   'id': '16782441-9775-46fe-b2f9-e7971473044f',
   'created_time': '2021-12-07T18:56:00.000Z',
   'last_edited_time': '2021-12-07T18:58:00.000Z',
   'cover': None,
   'icon': None,
   'parent': {'type': 'database_id',
    'database_id': 'aa1f8361-5ff9-4523-9302-887264317370'},
   'archived': False,
   'properties': {'ISIN': {'id': '%40%5BZb',
     'type': 'rich_text',
     'rich_text': []},
    'Tipo': {'id': 'IUV%7D',
     'type': 'select',
     'select': {'id': '4e0531e5-b001-4d03-ac9e-9054c79ee931',
      'name': 'Ingreso',
      'color': 'purple'}},
    'Descripción': {'id': 'MnyC',
     'type': 'rich_text',
     'rich_text': [{'type': 'text',
       'text': {'content': 'Flatex Deposit', 'link': None},
       'annotations': {'bold': False,
        'italic': False,
        'strikethrough': False,
        'underline': False,
        'code': False,
        'color': 'default'},
       'plain_text': 'Flatex Deposit',
       'href': Non